# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Севастопольский Артем Михайлович

Группа: 317

In [1]:
from functools import partial
import bisect
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial.distance import cdist
from sklearn import metrics, cross_validation, tree, ensemble
from sklearn.metrics import roc_auc_score
from joblib import Parallel, delayed
#from numba import jit

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
# data is transformed to np.array in order to manipulate it faster
data = data.values
X_train, X_test, y_train, y_test = train_test_split(data[:, 1:], data[:, 0],
                                                    test_size=0.3, random_state=241)
features = X_train.shape[1]

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния. Подсчитайте для каждой из них качество на тестовой выборке `X_test` при числе соседей $k = 10$. Метрика качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [7]:
class CategoricalDistanceCounter(object):
    def __init__(self, X_train):
        self.objects, self.features = X_train.shape
        self.counts = [pd.value_counts(X_train[:, i]) for i in xrange(X_train.shape[1])]
        # `counts[i]` is a Series that shows how many times value `i` appears in col № i.
        # `counts` depends on X_train.
        self.counts_dict = [cnt.to_dict() for cnt in self.counts]
        self.prob = [(cnt / len(X_train)).sort_values(inplace=False).to_dict() 
                     for cnt in self.counts]
        self.prob_vals = map(lambda d: np.array(d.values()), self.prob)
        self.prob_hit = [cnt * (cnt - 1) / self.objects / (self.objects - 1) 
                         for cnt in self.counts]
        # `prob_hit` is estimate of probability that two objects have the same feature value.
        self.prob_hit_cumsum = [np.hstack(([0], np.array(ar).cumsum()))
                                for ar in self.prob_hit]
    

#@jit
def dist(p, q, cntr, weights=None, dist_type=None):
    '''Accepts dist_type=1, 2 or 3.
    p, q and weights should be numpy vectors of the same length.
    '''
    dist = None
    if dist_type == 1:
        diff = (p != q).astype(int)
    elif dist_type == 2:
        p_prob = np.empty(cntr.features)
        for i in xrange(cntr.features):
            p_prob[i] = cntr.prob[i].get(p[i], 0)
        last_pos = np.empty(cntr.features)
        for i in xrange(cntr.features):
            last_pos[i] = bisect.bisect_right(cntr.prob_vals[i], p_prob[i])
        appr_prob = np.empty(cntr.features)
        for i in xrange(cntr.features):
            appr_prob[i] = cntr.prob_hit_cumsum[i][last_pos[i]]
        diff = np.where(p != q, 1, appr_prob)
        #p_prob = [cntr.prob[i].get(p[i], 0)
        #          for i in range(cntr.features)]
        #last_pos = [bisect.bisect_right(cntr.prob_vals[i], p_prob[i])
        #            for i in range(cntr.features)]
        #appr_prob = [cntr.prob_hit_cumsum[i][last_pos[i]]
        #             for i in range(cntr.features)]
        
    elif dist_type == 3:
        p_cnt = np.empty(cntr.features)
        q_cnt = np.empty(cntr.features)
        for i in xrange(cntr.features):
            p_cnt[i] = cntr.counts_dict[i].get(p[i], 0)
            q_cnt[i] = cntr.counts_dict[i].get(q[i], 0)
            # Taking care of possible troubles with logarithm.
            if p_cnt[i] == 0:
                p_cnt[i] = 1
            if q_cnt[i] == 0:
                q_cnt[i] = 1
        diff = (p != q).astype(int) * np.log(p_cnt) * np.log(q_cnt)
        #diff = (p != q).astype(int) * np.log([cntr.counts_dict[i].get(p[i], 0)
        #                                      for i in range(cntr.features)]) * \
        #    np.log([cntr.counts_dict[i].get(q[i], 0)
        #            for i in range(cntr.features)])
    else:
        raise Exception('Wrong metric')
    
    dist = np.sum(weights * diff)
    return dist

In [8]:
#@jit
def knn_predict(y_train, knn_idx, n_classes, K):
    '''Makes prediction by KNN method given matrix of distances from X_test to its neighbors.
    
    y_train: vector of answers for train set.
    knn_idx: 2D array that stores indices of K nearest neighbors 
    for each point of test set.
    n_classes: number of classes.
    K: number of neighbors.
    '''
    
    n_test_samples = knn_idx.shape[0]
    all_knn_idx = knn_idx[:, :K].ravel()
    all_knn_classes = y_train[all_knn_idx]
    knn_classes = all_knn_classes.reshape(n_test_samples, K)
    
    votes = np.empty((n_test_samples, n_classes))
    for cls in range(n_classes):
        votes[:, cls] = np.sum(knn_classes == cls, axis=1)
    y_pred = np.argmax(votes, axis=1)
    return y_pred

In [9]:
#@jit
def pdist(X_test, X_train, func):
    ans = np.empty((X_test.shape[0], X_train.shape[0]))
    for i in xrange(X_test.shape[0]):
        #print i,
        for j in xrange(X_train.shape[0]):
            ans[i, j] = func(X_test[i], X_train[j])
    print "pdist for chunk build."
    return ans

#@jit
def pdist_chunked(X_test, X_train, func, chunk_size=1000):
    ans = np.empty((X_test.shape[0], X_train.shape[0]))
    for i in range(0, X_test.shape[0], chunk_size):
        chunk = X_test[i:i + chunk_size]
        chunk_ans = pdist(chunk, X_train, func)
        ans[i:i + chunk_size] = chunk_ans
    return ans

#@jit
def pdist_for_chunk(func, start_idx, end_idx):
    ans = np.empty((end_idx - start_idx, nX_train.shape[0]))
    for i in xrange(start_idx, end_idx):
        #print i,
        for j in xrange(nX_train.shape[0]):
            ans[i - start_idx, j] = func(nX_test[i], nX_train[j])
    return ans

def pdist_parallel(func, n_samples=-1, chunk_size=1000,
                  n_jobs=1):
    if n_samples == -1:
        n_samples = nX_test.shape[0]
    
    ans = np.empty((n_samples, nX_train.shape[0]))
    with Parallel(n_jobs=n_jobs, verbose=1) as prl:
        chunk_ans = prl(delayed(pdist_for_chunk)(func, i, min(i + chunk_size, n_samples))
                        for i in range(0, n_samples, chunk_size))
    for i in range(0, n_samples, chunk_size):
        ans[i:i + chunk_size] = chunk_ans[i / chunk_size]
    return ans

#@jit
def get_knn_idx(pw_dist, K):
    '''Returns indices of K nearest neighbors
    '''
    return np.argpartition(pw_dist, K)[:, :K]

In [ ]:
dist_cntr = CategoricalDistanceCounter(X_train)
y_pred = []
for d in range(1, 4):
    print "Metric #{}. Preparing distance matrix...".format(d)
    #pairwise_dist = cdist(X_test, X_train, 
    #                      partial(dist_cntr.dist, weights=np.ones(features), dist_type=d))
    #pairwise_dist = pdist(X_test, X_train, 
    #                      partial(dist, cntr=dist_cntr, weights=np.ones(features), dist_type=d))
    
    nX_train, nX_test = X_train, X_test
    pairwise_dist = pdist_parallel(partial(dist, cntr=dist_cntr, weights=np.ones(features), dist_type=d),
                          n_samples=-1, chunk_size=500, n_jobs=19)
    knn_idx = get_knn_idx(pairwise_dist, 10)
    y_pred.append(knn_predict(y_train, knn_idx, 2, 10))
    
    print "prediction mean: ", y_pred[-1].mean()
    score = roc_auc_score(y_test, y_pred[-1])
    print "AUC-ROC score: ", score

Metric #1. Preparing distance matrix...
prediction mean:  0.972942732174
AUC-ROC score:  0.627809964051
Metric #2. Preparing distance matrix...


[Parallel(n_jobs=19)]: Done  20 out of  20 | elapsed:  7.2min finished


prediction mean:  0.974265079849
AUC-ROC score:  0.626747601395
Metric #3. Preparing distance matrix...


[Parallel(n_jobs=19)]: Done  20 out of  20 | elapsed: 34.7min finished


prediction mean:  0.975892584681
AUC-ROC score:  0.617014084325


[Parallel(n_jobs=19)]: Done  20 out of  20 | elapsed: 21.9min finished


#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

In [ ]:
#@jit
def knn_accuracy(y_train, y_test, pw_dist, K_opts):
    '''Calculates accuracy of KNN method on X_test.
    It runs KNN multiple times for all provided K_opts.
    
    K_opts: array_like of options for choosing K.
    '''
    # DataFrame for results
    acc = pd.DataFrame(index=K_opts)
    K_max = max(K_opts)
    knn_idx = get_knn_idx(pw_dist, K_max)
    
    for K in K_opts:
        print "Running KNN for K={}".format(K)
        y_pred = knn_predict(y_train, knn_idx, 2, K)
        cur_acc = roc_auc_score(y_test, y_pred)
        acc.ix[K, 'Accuracy'] = cur_acc
    return acc

In [ ]:
K_opts = np.full((7,), 2, dtype=float).cumprod()    # K_opts is powers of two: [2, 4, 8, 16, ..., 128]
dist_cntr = CategoricalDistanceCounter(X_train)
for d in range(1, 4):
    print "Metric #{}. Preparing distance matrix...".format(d)
    #pairwise_dist = cdist(X_test, X_train, 
    #                      partial(dist_cntr.dist, weights=np.ones(features), dist_type=d))
    nX_train, nX_test = X_train, X_test
    pairwise_dist = pdist_parallel(partial(dist, cntr=dist_cntr, weights=np.ones(features), dist_type=d),
                          n_samples=-1, chunk_size=900, n_jobs=11)
    print "Computing accuracy..."
    acc = knn_accuracy(y_train, y_test, pairwise_dist, K_opts)
    print(acc)
    print "Best K: {}, AUC-ROC score: {}".format(acc.idxmax()[0], acc.max()[0])

Metric #1. Preparing distance matrix...
Computing accuracy...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.571555
4    0.541972
8    0.524312
16   0.509517
32   0.506633
64   0.500000
128  0.500000
Best K: 2.0, AUC-ROC score: 0.57155461315
Metric #2. Preparing distance matrix...


[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed:  3.9min finished


Computing accuracy...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.561035
4    0.541798
8    0.528637
16   0.518096
32   0.511446
64   0.500829
128  0.500000
Best K: 2.0, AUC-ROC score: 0.561034500234
Metric #3. Preparing distance matrix...


[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed: 21.9min finished


Computing accuracy...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.565501
4    0.536189
8    0.519392
16   0.509571
32   0.503154
64   0.500000
128  0.500000
Best K: 2.0, AUC-ROC score: 0.565500682543


[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed: 14.2min finished


#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [12]:
#@jit
def counts(X_train, y_train, X_test):
    smooth_X_test = np.full_like(X_test, 0.5, dtype=float)
    counts = np.empty((X_test.shape[0],))
    clicks = np.empty((X_test.shape[0],))
    for col in xrange(features):
        vc = pd.value_counts(X_train[:, col])
        vc_dict = vc.to_dict()
        
        for i in xrange(X_test.shape[0]):
            counts[i] = vc_dict.get(X_test[i, col], 0)
        #counts = map(lambda el: vc.get(el, 0), X_test[col])
        
        vc_pos = pd.value_counts(X_train[y_train == 1, col])
        vc_pos_dict = vc_pos.to_dict()
        
        for i in xrange(X_test.shape[0]):
            clicks[i] = vc_pos_dict.get(X_test[i, col], 0)
        #clicks = map(lambda el: vc_pos.get(el, 0), X_test[col])
        
        smooth_X_test[:, col] = (clicks + 1).astype(float) / (counts + 2)
    
    return smooth_X_test

In [13]:
# Without folding.
smooth_X_train = counts(X_train, y_train, X_train)
smooth_X_test = counts(X_train, y_train, X_test)
print "Preparing distance matrix..."
#pairwise_dist = cdist(smooth_X_train, smooth_X_test, metric='euclidean')
def euclid_dist(a, b):
    return np.linalg.norm(a - b)

nX_train, nX_test = smooth_X_train, smooth_X_test
pairwise_dist = pdist_parallel(euclid_dist,
                               n_samples=-1, chunk_size=500, n_jobs=18)

K_opts = np.full((7,), 2).cumprod()    # K_opts is powers of two: [2, 4, 8, 16, ..., 128]
acc = knn_accuracy(y_train, y_test, pairwise_dist, K_opts)
print(acc)
print "Best K: {}, AUC-ROC score for counts without folding: {}".format(acc.idxmax()[0], acc.max()[0])

Preparing distance matrix...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.639422
4    0.633487
8    0.616911
16   0.620438
32   0.609793
64   0.609906
128  0.603673

[7 rows x 1 columns]
Best K: 2.0, AUC-ROC score for counts without folding: 0.639422091968


[Parallel(n_jobs=18)]: Done  20 out of  20 | elapsed:  5.3min finished


In [13]:
#@jit
def counts_folded(X_train, y_train, X_test, n_folds=3):
    print "Preparing folded data set with counts..."
    # We can use simple KFold:
    #kf = cross_validation.KFold(n=X_train.shape[0], n_folds=n_folds, shuffle=True, random_state=243)
    # or stratified KFold:
    cv = cross_validation.StratifiedKFold(y_train, n_folds=n_folds, 
                                          shuffle=True, random_state=243)
    
    fold_no = 0
    smooth_X_train = np.full_like(X_train, 0.5, dtype=float)
    for fold_index, rest_index in cv:
        # For fold of train set, counts and clicks are counted based on other parts.
        # For test set, counts and clicks are counted based on a whole train set.
        Xt_fold, Xt_rest = X_train[fold_index, :], X_train[rest_index, :]
        yt_rest = y_train[rest_index]
        smooth_Xt_fold = counts(Xt_rest, yt_rest, Xt_fold)
        smooth_X_train[fold_index, :] = smooth_Xt_fold
    
    smooth_X_test = counts(X_train, y_train, X_test)
    return smooth_X_train, smooth_X_test

In [14]:
# With folding.
smooth_X_train, smooth_X_test = counts_folded(X_train, y_train, X_test)
print "Preparing distance matrix..."
def euclid_dist(a, b):
    return np.linalg.norm(a - b)
nX_train, nX_test = smooth_X_train, smooth_X_test
#pairwise_dist = cdist(nX_test[:100], nX_train, 'euclidean')
pairwise_dist = pdist_parallel(euclid_dist,
                               n_samples=-1, chunk_size=500, n_jobs=18)

K_opts = np.full((7,), 2).cumprod()    # K_opts is powers of two: [2, 4, 8, 16, ..., 128]
acc = knn_accuracy(y_train, y_test, pairwise_dist, K_opts)
print(acc)
print "Best K: {}, AUC-ROC score for counts without folding: {}".format(acc.idxmax()[0], acc.max()[0])

Preparing folded data set with counts...
Preparing distance matrix...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.572065
4    0.532453
8    0.518204
16   0.509679
32   0.508129
64   0.504146
128  0.502488

[7 rows x 1 columns]
Best K: 2.0, AUC-ROC score for counts without folding: 0.572064633486


[Parallel(n_jobs=18)]: Done  20 out of  20 | elapsed:  6.7min finished


#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [106]:
#@jit
def paired(X):
    new_X = np.empty((X.shape[0], X.shape[1] + X.shape[1] * (X.shape[1] - 1) / 2), dtype=int)
    col_no = 0
    new_X[:, :features] = X.copy()
    for i in xrange(features):
        for j in xrange(i + 1, features):
            for k in xrange(X.shape[0]):
                new_X[k, features + col_no] = int(str(X[k, i]) + str(X[k, j]))
            col_no += 1
    return new_X

Xp_train = paired(X_train)
Xp_test = paired(X_test)

In [111]:
# Without folding.
smooth_Xp_train = counts(Xp_train, y_train, Xp_train)
smooth_Xp_test = counts(Xp_train, y_train, Xp_test)
print "Preparing distance matrix..."
def euclid_dist(a, b):
    return np.linalg.norm(a - b)
nX_train, nX_test = smooth_Xp_train, smooth_Xp_test
pairwise_dist = pdist_parallel(euclid_dist,
                               n_samples=-1, chunk_size=900, n_jobs=11)

K_opts = np.full((7,), 2).cumprod()    # K_opts is powers of two: [2, 4, 8, 16, ..., 128]
acc = knn_accuracy(y_train, y_test, pairwise_dist, K_opts)
print(acc)
print "Best K: {}, AUC-ROC score for counts without folding: {}".format(acc.idxmax()[0], acc.max()[0])

Preparing distance matrix...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.639422
4    0.633487
8    0.616911
16   0.620438
32   0.609793
64   0.609906
128  0.603673

[7 rows x 1 columns]
Best K: 2.0, AUC-ROC score for counts without folding: 0.639422091968


[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed:  4.1min finished


In [112]:
# With folding.
smooth_Xpf_train, smooth_Xpf_test = counts_folded(Xp_train, y_train, Xp_test)
print "Preparing distance matrix..."
def euclid_dist(a, b):
    return np.linalg.norm(a - b)
nX_train, nX_test = smooth_Xpf_train, smooth_Xpf_test
pairwise_dist = pdist_parallel(euclid_dist,
                               n_samples=-1, chunk_size=900, n_jobs=11)

K_opts = np.full((7,), 2).cumprod()    # K_opts is powers of two: [2, 4, 8, 16, ..., 128]
acc = knn_accuracy(y_train, y_test, pairwise_dist, K_opts)
print(acc)
print "Best K: {}, AUC-ROC score for counts without folding: {}".format(acc.idxmax()[0], acc.max()[0])

Preparing folded data set with counts...
Preparing distance matrix...
Running KNN for K=2.0
Running KNN for K=4.0
Running KNN for K=8.0
Running KNN for K=16.0
Running KNN for K=32.0
Running KNN for K=64.0
Running KNN for K=128.0
     Accuracy
2    0.572065
4    0.532453
8    0.518204
16   0.509679
32   0.508129
64   0.504146
128  0.502488

[7 rows x 1 columns]
Best K: 2.0, AUC-ROC score for counts without folding: 0.572064633486


[Parallel(n_jobs=11)]: Done  11 out of  11 | elapsed:  4.1min finished


## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [113]:
clf = tree.DecisionTreeClassifier(max_depth=1000, min_samples_leaf=6, random_state=243)
clf.fit(smooth_Xp_train, y_train)
yp_pred = clf.predict(smooth_Xp_test)
print "Score on Decision Tree: ", roc_auc_score(y_test, yp_pred)

Score on Decision Tree:  0.651982016661


#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [131]:
clf = ensemble.RandomForestClassifier(n_estimators=400, random_state=243,
                                     n_jobs=11)
clf.fit(smooth_Xp_train, y_train)
y_pred = clf.predict(smooth_Xp_test)
print "Score on Random Forest: ", roc_auc_score(y_test, y_pred)

Score on Random Forest:  0.684697718602


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [117]:
clf = ensemble.RandomForestClassifier(n_estimators=50000, random_state=243,
                                     n_jobs=11)
clf.fit(smooth_Xpf_train, y_train)
y_pred_c = clf.predict(smooth_Xpf_test)
print "Score on Random Forest using set of counts: ", roc_auc_score(y_test, y_pred_c)

Score on Random Forest using set of counts:  0.523396239436


Результат для выборки с парными признаками, к которым применены счетчики **с фолдингом**, оказался **ниже**. 

Возможные причины:
- при подсчете с фолдингом при обработке очередного фолда не учитываются данные из этого фолда, а это может быть важно. Кроме того, классы не сбалансированы: элементов класса 0 &mdash; 5%, класса 1 &mdash; 95% (см. выше). Поэтому при потере трети выборки могут теряться данные о закономерностях, дающих объекту метку "0".
- так как решающие деревья и случайные леса склонны к переобучению, то возможно, что оно возникает и здесь (на каждом фолде случайный лес слишком настраивается на закономерности из других фолдов).
- из-за того, что при фолдинге значения признаков считались независимо, признаки имеют немного разный смысл, и, возможно, из-за этого алгоритм не может выделить для них общие закономерности. К тому же, на фолдах обучающей выборки и на тестовой выборке счетчики вычисляются по разным размерам выборок.